# JupyterHub Notebook

### This notebook server is hosted on the OpenShift platform which provides a separate server for each individual user. The platform takes care of the provisioning of the server and allocating related to storage.

### First, install and import required libraries and watermark our file - to show what libraries and versions we're using. Then define utility functions to integrate with our Object storage and _Verta_ visualisation server.

In [ ]:
%pip install dill

# from alibi.explainers import AnchorTabular
# os.environ["MODIN_ENGINE"] = "ray"


In [ ]:
import os
import dill



## Load Lab parameters. Before running this cell, ensure you set the s3BucketFullPath value to your merged file. This is done in hyper_parameters.py

In [ ]:
from hyper_parameters import get_hyper_paras
user_id,PROJECT_NAME,EXPERIMENT_NAME,experiment_name,s3BucketFullPath = get_hyper_paras()

### Read the Merged Data

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
# import pandas as pd
# import modin.pandas as pd
from datetime import datetime
import watermark
import numpy as np
import pandas as pd
# import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from minio import Minio
from minio.error import ResponseError
import os
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline


# import tools as tools
%matplotlib inline
%load_ext watermark

In [ ]:
%watermark -n -v -m -g -iv


### In this next section, we initialise our variables and our Object Storage implemenation, Minio

In [ ]:
HOST = "http://mlflow:5500"

# PROJECT_NAME = "CustomerChurnUser60"
# EXPERIMENT_NAME = "CustomerChurnUser60"

os.environ['MLFLOW_S3_ENDPOINT_URL']='http://minio-ml-workshop:9000'
os.environ['AWS_ACCESS_KEY_ID']='minio'
os.environ['AWS_SECRET_ACCESS_KEY']='minio123'
os.environ['AWS_REGION']='us-east-1'
os.environ['AWS_BUCKET_NAME']='mlflow'

dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d%Y%H%M%S%f")
# experiment_name = "customerchurnuser29"
experiment_id = experiment_name + timestampStr

def get_s3_server():
    minioClient = Minio('minio-ml-workshop:9000',
                    access_key=os.environ['AWS_ACCESS_KEY_ID'],
                    secret_key=os.environ['AWS_SECRET_ACCESS_KEY'],
                    secure=False)

    return minioClient


import mlflow

# Connect to local MLflow tracking server
mlflow.set_tracking_uri(HOST)

# Set the experiment name...
mlflow.set_experiment(EXPERIMENT_NAME)

mlflow.sklearn.autolog(log_input_examples=True)





### In this next section, we pull in the merged CSV file prepared earlier by the data engineer. 

In [ ]:
minioClient = get_s3_server()
data_file = minioClient.fget_object("data", s3BucketFullPath, "/tmp/data.csv")
data_file_version = data_file.version_id
data = pd.read_csv('/tmp/data.csv')
data.head(5)


In [ ]:
# Convert binary variable into numeric so plotting is easier. We need to later take mean
data['Churn'] = data['Churn'].map({'Yes': 1, 'No': 0})

In [ ]:
data.replace(" ", np.nan, inplace=True)


In [ ]:
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'])

In [ ]:
mean = data['TotalCharges'].mean()
data.fillna(mean, inplace=True)
# Now we know that total charges has nan values
data.isna().sum()



In [ ]:
import category_encoders as ce
import joblib

names = ['gender', 'Partner', 'Dependents', 'PhoneService', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling']
# for column in names:
#     labelencoder(column)
data_enc = data
data_enc = data_enc.drop(['Churn', 'customerID'], axis=1)
data_enc.head(1)
enc = ce.ordinal.OrdinalEncoder(cols=names)
enc.fit(data_enc)
joblib.dump(enc, 'CustomerChurnOrdinalEncoder.pkl')
mlflow.log_artifact("CustomerChurnOrdinalEncoder.pkl")
labelled_set = enc.transform(data_enc)
labelled_set.tail(5)

In [ ]:

names = ['MultipleLines', 'InternetService', 'Contract', 'PaymentMethod', 'OnlineSecurity', 'OnlineBackup',
         'DeviceProtection', 'TechSupport']

ohe = ce.OneHotEncoder(cols=names)
data_ohe = data
data_ohe = data_ohe.drop(['Churn', 'customerID'], axis=1)
data_ohe.head(1)
ohe.fit(data_ohe)
joblib.dump(ohe, 'CustomerChurnOneHotEncoder.pkl')
mlflow.log_artifact("CustomerChurnOneHotEncoder.pkl")
final_set = ohe.transform(labelled_set)
final_set.tail(5)
labelled_set.shape

In [ ]:
labels = data['Churn']
X_train, X_test, y_train, y_test = train_test_split(final_set, labels, test_size=0.2)
print ('Training Data Shape',X_train.shape, y_train.shape)
print ('Testing Data Shape',X_test.shape, y_test.shape)

In [ ]:

Y = data['Churn']
X = final_set



### In this next section, we define the method train_and_save_model() where we train and then push our model to Verta - for visualisation 

In [ ]:
def train_and_save_model():
    kfold = KFold(n_splits = 3)
    model = DecisionTreeClassifier(max_depth=5, criterion='gini',min_samples_leaf = 3 ,min_samples_split = 10)
    model = model.fit(X_train, y_train)
    joblib.dump(model, 'CustomerChurnPredictor.sav')
    results = model_selection.cross_val_score(model,X,Y,cv = kfold)
    print(results)
    print('Accuracy',results.mean()*100)


    return model

### In this next section, we define the method explain_model(), where we make available an *_explanation_* of the reasons the model made the decisions it did. This is very useful for auditing purposes as well as for the Application development consumers of the model - who can optionally expand and utilise these reasons for their purposes.

In [ ]:
# from alibi.utils.data import gen_category_map

# def explain_model(model, X_train, X_test_record):
#     fnames = X_train.columns.tolist()
#     predict_fn = lambda x: model.predict_proba(x)
#     explainer = AnchorTabular(predict_fn, fnames)
#     explainer = explainer.fit(X_train.values, disc_perc=[25, 50, 75])
#     explanation = explainer.explain(X_test_record.values[0])
#     print('Anchor: %s' % explanation['anchor'])
#     print('Precision: %.2f' % explanation['precision'])
#     print('Coverage: %.2f' % explanation['coverage'])
#     return explainer

In [ ]:
model = train_and_save_model()
# explainer = explain_model(model, X_train, X_test)
# with open("CustomerChurnPredictorAlibi.dill", "wb") as x_f:
#     dill.dump(explainer, x_f)

In [ ]:
# minioClient = get_s3_server()
# minioClient.fput_object(bucket_name='models', object_name=experiment_id  +'/CustomerChurnPredictor.sav' , file_path='./CustomerChurnPredictor.sav')
# # minioClient.fput_object(bucket_name='models', object_name=experiment_id  +'/CustomerChurnPredictorAlibi.dill' , file_path='./CustomerChurnPredictorAlibi.dill')
# minioClient.fput_object(bucket_name='models', object_name=experiment_id  +'/CustomerChurnOrdinalEncoder.pkl' , file_path='./CustomerChurnOrdinalEncoder.pkl')
# minioClient.fput_object(bucket_name='models', object_name=experiment_id  +'/CustomerChurnOneHotEncoder.pkl' , file_path='./CustomerChurnOneHotEncoder.pkl')


## Test Models

In [ ]:
prediction = model.predict_proba([X_test.iloc[1]])
class_name = ['Not Churn', 'Churn']                                              
predicted_class =   np.argmax(prediction)                                    
print('Predicted Class name: ',class_name[predicted_class])
predicted_class_prob = np.max(prediction)
print('Predicted class Certainty: ', predicted_class_prob)


In [ ]:
print('Notebook complete!')